# Hierarchical clustering
Based on the keywords.

The data prepared in 01_Clean has been augmented with boolean keyword columns. Apply hierarchical clustering to these keywords, to identify structurein the dataset.

In [9]:
from scipy.cluster.hierarchy import linkage, fcluster
import pandas as pd
import os

In [10]:
DATA_PATH = 'data'  # path to folder with (downloaded or generated) data, will not be committed to git
ACRONYM = 'openalex_tng'  # Acronym, aka machine readable name, of this dataset ('tng' stands for "topic netework graph")

In [11]:
# load the data
clean_df = pd.read_csv(os.path.join(DATA_PATH, f"{ACRONYM}.csv.zip"), index_col=0)

In [40]:
# keyword columns
colnames = clean_df.columns[14: -34]
keywords_df = clean_df.loc[:, colnames]

### Clustering
1. Linkage method computes distance matrix,
2. fcluster assigns cluster labels to rows.

In [44]:
Z = linkage(keywords_df, method = 'ward', metric = 'euclidean')
clean_df['cluster_labels'] = fcluster(Z, 5, criterion='maxclust')

In [45]:
clean_df['cluster_labels']

id
https://openalex.org/W3016013581    4
https://openalex.org/W2111836958    1
https://openalex.org/W4286210710    2
https://openalex.org/W4285731179    1
https://openalex.org/W2136106200    3
                                   ..
https://openalex.org/W4380906871    5
https://openalex.org/W4388815328    2
https://openalex.org/W4388193111    2
https://openalex.org/W4393240927    1
https://openalex.org/W4391540009    4
Name: cluster_labels, Length: 65, dtype: int32

Continue here: https://campus.datacamp.com/courses/cluster-analysis-in-python/hierarchical-clustering-7e10764b-dd0d-4b0e-9134-513c3e750e68?ex=8